In [ ]:
import ipywidgets as widgets
import geopy
import gmaps
import time
import socket
import math
import threading
from IPython.display import display, clear_output, update_display

################################  Some Variables  ###################################

gmaps.configure(api_key='Enter API key')  #         Api-Key

server_address = ('192.168.1.25', 48569)  #         Server-Address and Port-Number

initial_location = (39.99515813942635, 32.750430237878376)  #        Initial-Location

length_tolerance = 3

angle_tolerance = 3

r = 6371                      #                         Radius of earth in kilometers

######################################################################################

fig = gmaps.figure(center=initial_location, zoom_level=20)

marker_locations = []  #                 Initialize the list to store marker locations

def handle_map_click(feature):
    global target_location
    lat, lng = feature.location
    target_location = (lat, lng)
    global marker_locations
    marker_locations.append(target_location)
    
drawing_layer = gmaps.drawing_layer()

fig.add_layer(drawing_layer)

drawing_layer.on_new_feature(handle_map_click)

display(fig)


Figure(layout=FigureLayout(height='420px'))

In [11]:
for i, location in enumerate(marker_locations, 1):
    print(f"Marker {i} Location:", location)

Marker 1 Location: (39.99557909199935, 32.752241866126425)
Marker 2 Location: (39.99562840776558, 32.752168105378516)
Marker 3 Location: (39.99567772349621, 32.75206215812243)
Marker 4 Location: (39.9957424503386, 32.75215201212443)
Marker 5 Location: (39.995834917149885, 32.7522727115301)


In [12]:
fig = gmaps.figure(center=marker_locations[0], zoom_level=20)

line_layer = gmaps.drawing_layer()

lines = [] 
dots = []   
hover_text = []

# Adding dots for each marker location
for i, location in enumerate(marker_locations):
    dot = gmaps.symbol_layer([location], fill_color='green', stroke_color='black', scale=[5])
    dots.append(dot)
    
    # Adding location info to hover_text
    hover_text.append(f"Location {i+1}: {location}")

# Adding lines between marker locations
for i in range(len(marker_locations)-1):
    line = gmaps.Line(
        start=marker_locations[i],
        end=marker_locations[i+1],
        stroke_weight=3.0,
        stroke_color="blue"  
    )
    lines.append(line)

# Adding the drawing layer, dots, and lines to the figure
fig.add_layer(drawing_layer)
#fig.add_layer(gmaps.drawing_layer(features=lines))  # Note: This line replaces the previous line_layer
fig.add_layer(gmaps.symbol_layer(marker_locations, fill_color='blue', stroke_color='yellow', scale=3, hover_text=hover_text))


In [13]:
# Creating a TCP client socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Connecting to the server

try:
    client_socket.connect(server_address)
    print("Connected to the server.")
except Exception as e:
    print(f"Unable to connect to the server: {e}")
    exit()

from math import radians, sin, cos, sqrt, atan2, asin, degrees

# the distance between two locations with following formula: acos(sin(lat1)*sin(lat2)+cos(lat1)*cos(lat2)*cos(lon2-lon1))*6371 

def dist(loc1, loc2):
    
    # Converting latitude and longitude from degrees to radians
    
    lat1, lon1 = radians(loc1[0]), radians(loc1[1])
    lat2, lon2 = radians(loc2[0]), radians(loc2[1])
    
    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a)) 
      
    return(c * r*1000)

marker_init = gmaps.symbol_layer([(39,32)])

def calculate_angle(point1, point2):
    # Calculation of the bearing angle between the line and the north
    lat1, lon1 = radians(point1[0]), radians(point1[1])
    lat2, lon2 = radians(point2[0]), radians(point2[1])

    delta_longitude = lon2 - lon1

    x = sin(delta_longitude) * cos(lat2)
    y = cos(lat1) * sin(lat2) - (sin(lat1) * cos(lat2) * cos(delta_longitude))

    bearing_angle_rad = atan2(x, y)
    bearing_angle_deg = (degrees(bearing_angle_rad) )

    return bearing_angle_deg

# Create display objects for each piece of text
print("#############################################")
output_display = display("", display_id=True)
print("#############################################")
latitude_display = display("", display_id=True)
longitude_display = display("", display_id=True)
angle_display = display("", display_id=True)
print("#############################################")
distance_display = display("", display_id=True)
def receive_data():
    while True:
        try:
            # Received data from the server
            data = client_socket.recv(1024).decode('utf-8')

            if not data:
                break  # Break the loop if no data is received

            # Splitting the received data into latitude and longitude
            try:
                latitude, longitude, angle = map(float, data.split(','))
            except ValueError:
                #update_display(f"Invalid data format received from the server.", display_id=output_display.display_id)
                
                continue

            # Update latitude, longitude, and angle displays
            update_display(f"Latitude: {latitude}", display_id=latitude_display.display_id)
            update_display(f"Longitude: {longitude}", display_id=longitude_display.display_id)
            

            # Process of the received data
            # Update the location for the initial_location which comes from the server 
            global initial_location
            global marker_locations
            initial_location = (latitude, longitude)
            global distance_first 
            distance_first = float('inf')
            global distance_final 
            distance_final = float('inf')
           
            #initial_location = (latitude, longitude)  
            
            # Calculate distances and print the minimum distance to the target and first location
            
            if len(marker_locations) > 0:
                distance_first = dist(initial_location, marker_locations[0])                
                for l in range(len(marker_locations)-1):
                    distances=[dist(marker_locations[l],marker_locations[l+1])]
                distance_final_sum=sum(distances)+distance_first
                update_display(f"Distance to the first location: {distance_first} m", display_id=distance_display.display_id)
                update_display(f"Minimum distance to the final location: {distance_final_sum} m", display_id=distance_display.display_id)
      
            marker_init.markers = [gmaps.Symbol(location=initial_location)]
            
            if(distance_final<length_tolerance) or len(marker_locations)==0:
                    update_display(f"You reached the target!", display_id=output_display.display_id)
                    break
            
            if distance_first<3 and len(marker_locations)!=0:
                marker_locations.pop(0)
              
            if len(marker_locations)>=1:
                line = gmaps.Line(
                    start=initial_location,
                    end=marker_locations[0],
                    stroke_weight=6.0,
                    stroke_color="red"  
                )
                line_layer.features = [line]
                
                #angle to the north which comes from map - line from our loc to the first loc
                
                turning_angle=calculate_angle(initial_location,marker_locations[0])
                update_display(f"Angle coming from AHRS: {angle}", display_id=angle_display.display_id)
                
                
                #angle we need to turn to reach the target
                
                turning_angle=turning_angle-angle
                
                if(angle_tolerance>abs(turning_angle)):
                    update_display(f"Go straight!", display_id=output_display.display_id)
                elif (turning_angle >= angle_tolerance) and (turning_angle <= (180-angle_tolerance)):
                    update_display(f"Return LEFT at angle: {turning_angle} ", display_id=output_display.display_id)
                elif (turning_angle > 180):
                    update_display(f"Return RIGHT at angle: {360-turning_angle} ", display_id=output_display.display_id)
                elif (turning_angle >=(-180)) and (turning_angle < -abs(angle_tolerance)):
                    update_display(f"Return RIGHT at angle: {abs(turning_angle)} ", display_id=output_display.display_id)
                elif turning_angle < -180:
                    update_display(f"Return LEFT at angle: {360 + turning_angle} ", display_id=output_display.display_id)
                else:
                    continue
        
            
        except Exception as e:
            print(f"Error receiving data from server: {e}")
            break

        time.sleep(1)


# Start the receive_data function in a separate thread

receive_thread = threading.Thread(target=receive_data)
receive_thread.start()
# Add the drawing layer

lines.append(line)

fig.add_layer(line_layer)
fig.add_layer(marker_init)
fig.add_layer(gmaps.drawing_layer(features=lines)) 
fig




Connected to the server.
#############################################


'You reached the target!'

#############################################


'Latitude: 39.9958349171'

'Longitude: 32.7522718203'

'Angle coming from AHRS: 57.35455571'

#############################################


'Minimum distance to the final location: 14.616637833580647 m'

Figure(layout=FigureLayout(height='420px'))